
# General information of the datascrapping process.

Here, we get vital data for the key airports we want to scrape data from:

location (coordinates, country, city), IATA code,

This then allows us, using the wikipedia API to find each airport's corresponding wikipedia link and wikipedia name.
For example JFK is encoded in wikipedia's internal database as John_F._Kennedy_International_Airport. This matches the last part of the url in the wikipedia page link for the airport:     
https://en.wikipedia.org/wiki/John_F._Kennedy_International_Airport

For each time range:

The wikipedia page's raw text can be scrapped predictiably for a list of destinations from an airport. We will encode airport destinations using iata code.

We do this for each airport, generating a large csv file of airport-destination pairs for a particular time change.

We will look at 2 time ranges (now(as of June 4th 11am Eastern Time), before Jan 1, 2000 UTC 0)

### Sources :


list of top 1000 airports by traffic to scrape:

https://gettocenter.com/airports/top-100-airports-in-world/1000#google_vignette  


detailed airport database to cross reference:

https://www.partow.net/miscellaneous/airportdatabase/index.html#Downloads 

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
# Suppress just SettingWithCopyWarning
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

# Part 1: get basic data for the list of top 1000 airports 

In [2]:
# Example: load HTML from a URL or string
url = "https://gettocenter.com/airports/top-100-airports-in-world/1000"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

table = soup.find("table")  
rows = table.find_all("tr")

with open("./data/top_airports_basic_data.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    for row in rows:
        # Extract all cells (td or th)
        cols = row.find_all(["td", "th"])
        # Write the row text content to CSV
        writer.writerow([col.get_text(strip=True) for col in cols])
    f.close()

after replacing blank strings "", read in again csv file and add column names

In [3]:
data = pd.read_csv("./data/top_airports_basic_data.csv", names=["full_name", "iata", "city", "country", "estimated_pax"])
print(len(data))
data=data.dropna()
print(len(data))
data.head(n=11)

1087
989


,full_name,iata,city,country,estimated_pax
1.0,Hartsfield–Jackson Atlanta International Airport,ATL,Atlanta,United States,"103,902,992"
2.0,Beijing Capital International Airport,PEK,Beijing,China,"95,786,442"
3.0,Dubai International Airport,DXB,Dubai,United Arab Emirates,"88,242,099"
4.0,Los Angeles International Airport,LAX,Los Angeles,United States,"84,557,968"
5.0,O'Hare International Airport,ORD,Chicago,United States,"79,828,183"
6.0,Heathrow Airport,LHR,London,United Kingdom,"78,014,598"
7.0,Haneda Airport,HND,Tokyo,Japan,"76,476,251"
8.0,Hong Kong International Airport,HKG,Hong Kong,Hong Kong,"72,665,078"
9.0,Shanghai Pudong International Airport,PVG,Shanghai,China,"70,001,237"
10.0,Charles de Gaulle International Airport,CDG,Paris,France,"69,471,442"


we now have obtain basic information for the top airports the the world. save the pdf

In [4]:
data.to_csv("./data/top_airports_basic_data.csv")

#  Part 2:
Get wikipedia urls and for each iata code by using search api in wikipedia

In [ ]:
#testing a function
def get_wikipedia_url_from_name(name):
    """Search Wikipedia using IATA code and return the best-matching article title."""
    search_url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "list": "search",
        "srsearch": f"{name}",
        "format": "json"
    }

    response = requests.get(search_url, params=params)
    data = response.json()
    try:
        raw_name = data['query']['search'][0]['title'] 
        raw_name = raw_name.replace(" ", "_") #replace the raw name spaces with _

        return "https://en.wikipedia.org/wiki/"+raw_name #format for english wiki
    except (KeyError, IndexError):
        return None
    
print(get_wikipedia_url_from_name("Hartsfield–Jackson Atlanta International Airport")) #popular airport
print(get_wikipedia_url_from_name("Gobernador Castello Airport")) #more obscure airport


https://en.wikipedia.org/wiki/Hartsfield–Jackson_Atlanta_International_Airport
https://en.wikipedia.org/wiki/Gobernador_Edgardo_Castello_Airport


now, run for all airports

In [6]:
data = pd.read_csv("./data/top_airports_basic_data.csv")

In [ ]:
data["wiki_url"] =None #add column
for index, row in data.iterrows():
    try:
        print("current index:", index)
        name = row["full_name"]
        url = get_wikipedia_url_from_name(name)
        data["wiki_url"][index] = url
    except:
        data["wiki_url"][index] = None

current index: 0
current index: 1
current index: 2
current index: 3
current index: 4
current index: 5
current index: 6
current index: 7
current index: 8
current index: 9
current index: 10
current index: 11
current index: 12
current index: 13
current index: 14
current index: 15
current index: 16
current index: 17
current index: 18
current index: 19
current index: 20
current index: 21
current index: 22
current index: 23
current index: 24
current index: 25
current index: 26
current index: 27
current index: 28
current index: 29
current index: 30
current index: 31
current index: 32
current index: 33
current index: 34
current index: 35
current index: 36
current index: 37
current index: 38
current index: 39
current index: 40
current index: 41
current index: 42
current index: 43
current index: 44
current index: 45
current index: 46
current index: 47
current index: 48
current index: 49
current index: 50
current index: 51
current index: 52
current index: 53
current index: 54
current index: 55
cu

In [ ]:
data.head(n=989)

finding number of null entries in the wiki_url column

In [ ]:
print(f"Number of null entries: {data['wiki_url'].isnull().sum()}")

find the null rows

In [ ]:
data[data['wiki_url'].isnull()]

saving the updated data

In [ ]:
data.to_csv("./data/top_airports_basic_data.csv")

# Part 3:

Try to get wikipedia airport names with underscores to make querying easier, and testing a script to get a list of destinations from a wikipedia article

### getting wikipedia name


Populating wikipedia names in our top_airports_basic_data.csv

In [ ]:
#the wikipedia name based on the url, is simply found as:
print("name:", "https://en.wikipedia.org/wiki/Victoria_International_Airport".split("https://en.wikipedia.org/wiki/")[1])

In [ ]:
data = pd.read_csv("./data/top_airports_basic_data.csv")
data.head(n=1)

In [ ]:
data["wiki_name"] = None 
wiki_names = []

for i, row in data.iterrows():
    url = str(row["wiki_url"])
    wiki_names.append(url.split("https://en.wikipedia.org/wiki/")[1])
    
data["wiki_name"] = wiki_names
#overwrite
data.to_csv("./data/top_airports_basic_data.csv")

# Part 4 : Transition into getting routes data (in the other jupyter notebook)

### testing script to get raw text from a wikipedia page and get a list of destination-tuples for the current destinations listed in wikipedia

In [ ]:
#function to extract text between two found substrings
def extract_between(text, start, end):
    start_idx = text.find(start)
    if start_idx == -1:
        return "" #return empty string
    start_idx += len(start)
    end_idx = text.find(end, start_idx)
    if end_idx == -1:
        return ""
    return text[start_idx:end_idx]
#function to get iata code from wikipedia name
def get_iata_code(iata,ref_data=pd.read_csv("./data/top_airports_basic_data.csv")):
    matched = ref_data[ref_data["wiki_name"]==iata].iloc[0]
    return matched["iata"]
    

def get_destinations(iata_source, wiki_name):
    search_url = f"https://en.wikipedia.org/w/index.php?title={wiki_name}&action=raw"
    response = requests.get(search_url)
    full_text = response.text
    #get to the passenger destinations section marked by a predictiable substrings ()
    # Call both and choose the first non-None result
    destinations_text = extract_between(full_text, "=== Passenger ===", "==") or extract_between(full_text, "===Passenger===", "==")
    destinations_list = destinations_text.split("<!-- -->")#split predictably
    
    return_list = [] #return a list of destination of tuple of the form (iata_source, starting_wiki_name, dest_wikipedia_name,airline, isSeasonal)
    #ignore the first part, since that is some disclaimer
    for i in range(1, len(destinations_list)):
        airlineDest = destinations_list[i]#we get a list relating to airlines and their list of destinations
        #get information before and the seasonal destination section (if present)
        beforeSeasonal = ""
        afterSeasonal = None
        if "'''Seasonal:'''" in airlineDest:
            seasonalSplit = airlineDest.split("'''Seasonal:'''")
            beforeSeasonal = seasonalSplit[0]
            afterSeasonal = seasonalSplit[1]
        else: 
            beforeSeasonal = airlineDest 
        
        #get the airline based on a predictable pattern (in the first element)
        beforeSeasonalsplit = beforeSeasonal.split(",") #split based on commas
        airline_firstdest = beforeSeasonalsplit[0].split(" | [[")  #split between the airline and first destination
        airline = str(airline_firstdest[0]) #extract the airline
        airline = extract_between(airline,"[[","]]") #remove the [[]]
        #do a final split just in case (even if | is not found), [0] selects the current string
        firstdest = ""  #get first destination (assuming not seasonal) 
        if len(airline_firstdest) == 2:
            destination = airline_firstdest[1].split("|")[0].strip()
            destination_wiki_name = destination.replace(" ", "_") #replace destination to get the destination iata_code
            return_list.append((iata_source,wiki_name,destination_wiki_name,airline,False )) #(iata_source, starting_wiki_name, dest_wikipedia_name,airline, isSeasonal)
        #loop thorugh the rest of the comma list for the before seasonal split if possible 
        for i in range(1, len(beforeSeasonalsplit)):
            destination = beforeSeasonalsplit[i]
            extracted_destination = extract_between(destination,"[[","]]")
            if (extracted_destination!=""): #assuming 
                destination_wiki_name = extracted_destination.split("|")[0].strip()
                destination_wiki_name = destination_wiki_name.replace(" ", "_")
                return_list.append((iata_source,wiki_name,destination_wiki_name,airline,False )) #(iata_source, starting_wiki_name, dest_wikipedia_name,airline, isSeasonal)
        #loop through the seasonal destinations list if the current airline has it
        if (afterSeasonal):
            afterSeasonalSplit = afterSeasonal.split(",")
            for i in range(0, len(afterSeasonalSplit)):
                destination = afterSeasonalSplit[i]
                extracted_destination = extract_between(destination,"[[","]]")
                if (extracted_destination!=""): #assuming 
                    destination_wiki_name = extracted_destination.split("|")[0].strip()
                    destination_wiki_name = destination_wiki_name.replace(" ", "_")
                    return_list.append((iata_source,wiki_name,destination_wiki_name,airline,True )) #(iata_source, starting_wiki_name, dest_wikipedia_name,airline, isSeasonal)
            
    return return_list


testing on JFK airport data

In [ ]:

get_destinations("JFK","John_F._Kennedy_International_Airport")

In [ ]:
get_destinations("PAP","Toussaint_Louverture_International_Airport")